In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os.path as path

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.preprocessing as pre
# import librosa
import music21 as mus

import vmo
import vmo.analysis as van
import vmo.generate as vge
# Music analysis
import vmo.utils.chromagram as vchroma
import vmo.utils.music21_interface as vmusic
import vmo.utils.harmonic_changes as vharm
# Logics
import vmo.logics.model_checking as vmodel

In [ ]:
## Midi file access
midi_path = path.expanduser('~/Musique/midi/')
titles = ['blues_for_alice', 'marmaduke',
          'now_s_the_time', 'ornithology']
def make_filename(title):
    midi_file = 'charlie_parker/omnibook/' + title + '.mid'
    return midi_path + midi_file
    
"""Oracle generation."""

# Generate music21 stream from MIDI file

# Choose whether all pieces should be concatenated as a corpus
concatenate = False

if concatenate:
    # Initialize new, empty stream
    notes = mus.stream.Stream()
    for title in titles:
        # Aggregate all four titles
        stream_full_band = mus.converter.parse(make_filename(title))
        stream_piano = stream_full_band[2]
        flat_piano = stream_piano.flat.notesAndRests.quantize()
        
        duration = flat_piano.duration.quarterLength
        delta_quadratura = duration % 4
        flat_piano.append(mus.note.Rest(delta_quadratura))
        
        notes.append(flat_piano.flat.notesAndRests.quantize())
    notes = notes.flat.notesAndRests.quantize()
    chosen_title = 'Concatenation'
else:
    chosen_title = titles[3]  # Choose among the proposed titles
    stream_full_band = mus.converter.parse(make_filename(chosen_title))
    stream_piano = stream_full_band[2]
    notes = stream_piano.flat.notesAndRests.quantize()

In [ ]:
# Generate VMO from music21 stream
oracle = vmusic.from_stream(notes, framesize=4.0, dfunc='tonnetz', threshold=2)

In [ ]:
notes.flat.notesAndRests.show('text')
chosen_title
# notes.show('midi')

In [ ]:
"""Harmonic changes detection and chord progression extraction."""

# Extract the first 16 quarter-notes
_beginning_notes = vmusic.extract_frame(notes, 0., 32.)
# Compute offsets of harmonic changes (in quarter-length)
harmonic_changes = vharm.from_stream_by_offsets(
    _beginning_notes, framesize=1/4.)
# Compute the chord progression in the extracted frame
prog = vmusic.get_chord_progression(_beginning_notes)

print("First two measures:")
_beginning_notes.show('text')
print("\nExtracted progression:\n" + str(prog))
print("\nHarmonic changes:\n" + str(harmonic_changes))

In [ ]:
# Extract one frame from the oracle using the original stream
vmusic.extract_frame_oracle(notes.flat, oracle, 6).flat.show('text')

In [ ]:
"""Integrated MIDI playback.

Might need some parameters-tweaking to work if midi player binding is not
done straight out-of-the-box with music21.

In that case, run:
>>> from music21 import *
>>> us = environment.UserSettings()
>>> us.create()
>>> us['midiPath'] = <PATH/TO/MIDI/PLAYER>
>>> # Example : us['midiPath'] = u'/usr/bin/totem-audio-preview'
"""

# _beginning_notes.show('midi')

In [ ]:
"""Chord-progression extraction."""
# Two sequences of degrees. We will try to generate a path following the first
# then the second (possibibly with a non-empty arbitrary path between both) in `oracle`

prog_1 = [[1, 4, 7, 3, 6, 7, 3, 6]]
prog_2 = [[1, 4, 2, 5, 1, 6, 2, 5]]

gen_path_1, chosen_tonic_1 = vmodel.make_progression_from_degrees(
    oracle, prog_1, original_stream=notes, enable_motions=True)
gen_path_2, chosen_tonic_2 = vmodel.make_progression_from_degrees(
    oracle, prog_2, original_stream=notes, enable_motions=True)

def path_printer(path, tonic):
    if path:
        print("A path has successfully been generated!")
        print("The chosen tonic is: " + tonic)
        print("\nThe path is the following: \n")
        for state in path:
            print(state)
    else:
        print("No path was found.")

print("Path 1\n")
path_printer(gen_path_1, chosen_tonic_1)

print("\n===\n\nPath 2\n")
path_printer(gen_path_2, chosen_tonic_2)


In [ ]:
def midiSave(stream, filename):
    mf = mus.midi.translate.streamToMidiFile(stream)
    mf.open(filename + '.mid', 'wb')
    mf.write()

# Recompose new stream from extracted path

if gen_path_1:
    extracted_1 = vmusic.path_to_stream(notes, gen_path_1[0:-1], framesize=4.0).quantize()
    midiSave(extracted_1, chosen_title + '_genpath_1')
    print(vmusic.get_chord_progression(extracted_1, framesize=4.0))
    
    # Plot piano roll of the extracted stream
    extracted_1.plot('pianoroll')
    
if gen_path_2:
    extracted_2 = vmusic.path_to_stream(notes, gen_path_2[0:-1], framesize=4.0).quantize()
    midiSave(extracted_2, chosen_title + '_genpath_2')
    print(vmusic.get_chord_progression(extracted_2, framesize=4.0))
    
    # Plot piano roll of the extracted stream
    extracted_2.plot('pianoroll')

In [ ]:
extracted_1.show('midi')

In [ ]:
extracted_2.show('midi')